In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [14]:
# URL of the website
URL = "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"
# Set up the Chrome driver
driver = webdriver.Chrome()
driver.get(URL)
driver.maximize_window()
time.sleep(5)  # Wait for the page to load

In [15]:
# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('sb_bus_details.csv', index=False)

# Close the driver
driver.quit()

In [16]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,SBSTC-BURDWAN - KOLKATA - 05:45 (BWN DEPOT) - 2,Non AC Seater (2+3),05:45,02h 00m,07:45,4.3,INR 95,46 Seats available
1,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,SBSTC-BURDWAN - KARUNAMOYEE - 05:50 (BWN DEPOT...,Non AC Seater (2+3),05:50,02h 30m,08:20,4.1,INR 100,40 Seats available
2,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,SBSTC-BURDWAN - KOLKATA - 06:15 (BWN DEPOT) - 4,Non AC Seater (2+3),06:15,02h 00m,08:15,3.9,INR 95,37 Seats available
3,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,SBSTC-BURDWAN - KARUNAMOYEE - 06:25 (BWN DEPOT...,Non AC Seater (2+3),06:25,02h 30m,08:55,4.3,INR 100,28 Seats available
4,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...,SBSTC-BURDWAN - KARUNAMOYEE - 07:10 (BWN DEPOT...,Non AC Seater (2+3),07:10,02h 30m,09:40,4.1,INR 100,30 Seats available
...,...,...,...,...,...,...,...,...,...,...
1090,Purulia to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/purulia-to-d...,SBSTC-PURULIA - MALDA - 05:00 (PRL DEPOT) - 2528,Non AC Seater (2+3),05:00,03h 40m,08:40,2.6,INR 112,52 Seats available
1091,Purulia to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/purulia-to-d...,SBSTC-PURULIA - BERHAMPUR - 05:30 (PRL DEPOT) ...,Non AC Seater (2+3),05:30,04h 00m,09:30,3.5,INR 112,52 Seats available
1092,Purulia to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/purulia-to-d...,,,,,,,,
1093,Purulia to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/purulia-to-d...,,,,,,,,
